# Sample for KFServing SDK v1beta1

This is a sample for KFServing SDK v1beta1. 

***!!!***

***Kfserving depends on azure-storage-blob 2.1.0.***

***This will break some of the ydata packages.***

***Please check the kfserving from pipelines laboratory.***

In [ ]:
!pip install azure-storage-blob==2.1.0

### Input variables


In [ ]:
inferenceServiceName = "randomforest1"
inFerenceServiceStorageUri = "gs://pipelines_artifacts/kfserving/sklearn/randomforest"
inputFile = "https://raw.githubusercontent.com/ydataai/academy/master/tutorials/Model%20Serving/KfServing/input_files/input.json"


In [ ]:
from kubernetes import client 
from kfserving import *


## Define InferenceService

Define the inferenceservice basic on the endpoint spec.

In [ ]:
api_version = constants.KFSERVING_GROUP + "/v1beta1"
namespace = utils.get_default_target_namespace()

annotations = {}
annotations["sidecar.istio.io/inject"] = "false"
isvc = V1beta1InferenceService(
    api_version=api_version,
    kind=constants.KFSERVING_KIND,
    metadata=client.V1ObjectMeta(
        name=inferenceServiceName, namespace=namespace, annotations=annotations
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            min_replicas=1,
            sklearn=V1beta1SKLearnSpec(
                # Store your model with the name model.joblib and provide the file location
                # to the storage_uri variable
                storage_uri=inFerenceServiceStorageUri,
            ),
        )
    ),
)


## Create the InferenceService

In [ ]:
KFServing = KFServingClient()
KFServing.create(isvc)


In [ ]:
KFServing.wait_isvc_ready(inferenceServiceName, namespace=namespace)


## Test the InferenceService

In [ ]:
import json
import requests

response = requests.get(inputFile)
data = response.text

url = (
    "http://"
    + inferenceServiceName
    + "-predictor-default"
    + "."
    + namespace
    + ".svc.cluster.local/v1/models/"
    + inferenceServiceName
    + ":predict"
)

print(url)
response = requests.post(url, json.dumps(json.loads(data)))

print("Input: ")
print(json.dumps(data))
print("\n")
print("Output: ")
print(response.text)


## Delete the InferenceService

In [ ]:
KFServing.delete(inferenceServiceName, namespace=namespace)
